# Libraries

In [ ]:
import jenkspy
import os
from kneed import KneeLocator
import math
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import scipy.stats as stats

# Common Functions

In [ ]:
def get_id_list(n):
  id_list = []
  for i in range(1, n+1):
    id_list.append('P'+str(i))
  return id_list

# Location

In [ ]:
# loc_root = 'E:/UVA One Drive-Asus/OneDrive - University of Virginia/'
bool_number_of_change_cal = True # Make it true when you are 
loc_root = 'C:/Users/wyd2hu/OneDrive - University of Virginia/'
loc_root_features_tiles_18 = loc_root+"Hridita/Rhythms/Findings/Tiles18_HeartRatePPG.xlsx"

# Identifying the changes in Rhythm 

In [ ]:
def visualize_by_scatter(x_data, y_data,):
    
    bar_fig = go.Scatter(x=x_data, y=y_data)
    
    layout = go.Layout(
      title=go.layout.Title(text='', x=0),
      xaxis=go.layout.XAxis(title=go.layout.xaxis.Title(
      text="Participant", font=dict(family='Times New Roman', size=18, color='#000000')),
      tickmode = 'array', color="#000000", tickfont=dict(family='Times New Roman', size=18)),

      yaxis=go.layout.YAxis(title=go.layout.yaxis.Title(text= 'Break point',
      font=dict(family='Times New Roman', size=18, color='#000000')),
      tickmode = 'array', color="#000000", tickfont=dict(family='Times New Roman', size=18)),
      legend=dict(orientation="h", x=0.3, y=1.1,  font=dict(family='Times New Roman', size=18), bgcolor='#FFFFFF'),
    
      plot_bgcolor = '#FFFFFF')
    fig = go.Figure(bar_fig, layout=layout)
    # fig.add_shape(type="line", x0=4, y0=y_min, x1=4, y1=y_max, line=dict(color='#000000', width=0.8, dash="dashdot"))
    fig.show()

In [ ]:
df_rhy_tiles18 = pd.ExcelFile(loc_root_features_tiles_18)
id_list = df_rhy_tiles18.sheet_names

print('Should be equal', len(id_list), len(set(id_list)))
exclusion_list_tiles_18 = ['91297102-4775-4ea1-a10d-4',  'e3e5e4aa-5950-4f1f-915c-c', 
                        '41f2fcc5-e1a6-42d0-9b51-6',  'f30085f9-fdb0-49fb-b242-2',  
                        'fb3a4dd7-3cb5-438e-a75d-2',  '32de9693-b5b6-4292-addd-2',  
                        '24166136-6ee3-4521-abaf-9',  '1b1bcdb5-00eb-4a8c-8c05-a',  '2d05ec74-7279-4b26-b3ec-d',  
                        '3bdb1a29-023b-4fb5-8f49-6',  'a8800ef0-619c-4a3c-af5b-d']
exclusion_list_tiles_19 = ['4f91d234-b726-497b-aa81-d', '6df6f668-c75c-4264-9e97-3', 'd34aefad-d744-4462-8a77-c' 
                           '89209220-8ecf-4cd6-a9a4-a', '8dde8abc-014e-480a-9a07-c', '89209220-8ecf-4cd6-a9a4-a',
                          'a3df0da8-33d9-463b-babd-5', 'a9b3538d-491a-4615-ad89-c', 'ce871d21-38ba-4875-882f-0',
                          'd34aefad-d744-4462-8a77-c', 'ebf4f216-2e6a-4dd2-9266-e']

if 'Tiles18' in loc_root_features_tiles_18:
    change_file_name = 'tiles_18_n_change.xlsx'
    exclusion_list = exclusion_list_tiles_18
    list_breakpoint_range = list(range(2, 21))
else:
    change_file_name = 'tiles_19_n_change.xlsx'
    exclusion_list = exclusion_list_tiles_19
    list_breakpoint_range = list(range(2, 9))
print(change_file_name)

data_list_n_change = []
all_parti_threshold_x = []
all_parti_threshold_y = []

df_violin = pd.DataFrame()

for stable_breakpoint in [12]:
    single_violin_data = []
    for p_id in id_list:

        # if '2d05ec74-7279-4b26-b3ec-d' in p_id:
        if p_id not in exclusion_list: #   0e33f78a-7064-44ff-a8a4-7, 14eb688e-385b-4d6c-8cc1-0,  16812063-e5df-4657-b86c-5
            print(stable_breakpoint, p_id)
        
            threshold_x = []
            threshold_y = []

            df_rhy_tiles18 = pd.read_excel(loc_root_features_tiles_18, p_id)
            df_rhy_tiles18 = df_rhy_tiles18.iloc[1:] # first row (0) contains NaN values. Not an issue, BTW.
            first_time = df_rhy_tiles18.iloc[0]["N_Times"] # N_Times column contains the data of the nth (N) week
            last_time = df_rhy_tiles18.iloc[-1]["N_Times"]
    
            # print(p_id, 'DF size', df_rhy_tiles18.shape)
    
            step =  3 # steps to move. e.g., if it is 3, we are inferring changes of next 6 days (3: in each point, there is data of 2 days)
            last_index = stable_breakpoint # how many weeks' data are used for estimtaing rhythm as baseline

            if bool_number_of_change_cal:
                n_times = math.ceil((last_time - last_index) / step) + 1 # +1: let's say, there are 10 instances. Then, with a step of 3 and initial instance of 7 should make the total number of iterations 2
            else:
                n_times = math.ceil((last_time - last_index) / step) # ceil: e.g., 10 instances, intial 5 and step 4. Then, we need 5+4, 5+4 + 1
            n_counter = last_index + 1 # 2 means week after 8 days.

            for i_time in range(1, n_times+1):
                threshold_x.append(i_time) # data in each iteration

                if bool_number_of_change_cal:
                    df_sub_rhy = df_rhy_tiles18.iloc[list(range(0, stable_breakpoint))].copy() # can not be last_index+1 since you are starting from 0 and there are df_rhy_tiles18.shape[0]-1 values.
                else:
                    df_sub_rhy = df_rhy_tiles18.iloc[list(range(0, last_index))].copy() # can not be last_index+1 since you are starting from 0 and there are df_rhy_tiles18.shape[0]-1 values.
                
                if df_sub_rhy.shape[0] > 1: # if there is a week (after getting the sub dataframe df_sub_rhy) where there is just 1 instance, we can not move since we will not be able to find out the knee
                        break_points = jenkspy.jenks_breaks(df_sub_rhy["correct_acrophase"], n_classes=2)
                        print(p_id, 'DF size', df_rhy_tiles18.shape, break_points[1])
                        threshold_y.append(break_points[1])
                        
                        if ((last_index + step) <= df_rhy_tiles18.shape[0]): # just checking whether there are 3 (step) elements next to be retrieved
                            df_next_n_rhythm = df_rhy_tiles18.iloc[list(range(last_index, last_index+step))].copy()
                            last_index += step
                        else: # otherwise, we are taking all the remaining values
                            df_next_n_rhythm = df_rhy_tiles18.iloc[list(range(last_index, df_rhy_tiles18.shape[0]))].copy()
                            last_index = df_rhy_tiles18.shape[0]
                        
                        n_change = df_next_n_rhythm[df_next_n_rhythm['correct_acrophase'] > (break_points[1] + 0.1)].shape[0]
                        n_change += df_next_n_rhythm[df_next_n_rhythm['correct_acrophase'] < (break_points[1] - 0.1)].shape[0]
                        data_list_n_change.append([n_counter, n_change, break_points[1], p_id])
                        
                        n_counter += 1
                    # except Exception as e:
                    #     print('Problem 😒', df_sub_rhy.shape[0], df_sub_rhy.isna().sum().sum(), p_id)
                    #     print(e)
                    #     print(sorted(df_sub_rhy["correct_acrophase"]))
    
            # visualize_by_scatter(threshold_x, threshold_y)
            single_violin_data.append(np.mean(list(map(abs, threshold_y)) - abs(threshold_y[0])))
            all_parti_threshold_x.append(threshold_x)
            all_parti_threshold_y.append(threshold_y)            
    df_violin['S '+str(stable_breakpoint)] = single_violin_data

# Saving the data on the number of changes
df_n_change = pd.DataFrame(data=data_list_n_change, columns = ['n_counter', 'n_change', 'break_point','p_id'])
df_n_change.to_excel(loc_root + 'Hridita/Rhythms/Findings/Jenks/'+change_file_name, index=False)

In [ ]:
print(sum(df_n_change['n_change']))

# Understanding the breakpoint for each P

In [ ]:
def visualize_by_bar_chart(x_data, y_data, y_axis):    
    bar_fig = go.Bar(x=x_data, y=y_data)
    print(np.mean(y_data))
    
    layout = go.Layout(
      title=go.layout.Title(text='', x=0),
      xaxis=go.layout.XAxis(title=go.layout.xaxis.Title(
      text="Participant", font=dict(family='Times New Roman', size=18, color='#000000')),
      tickmode = 'array', color="#000000", tickfont=dict(family='Times New Roman', size=18)),

      yaxis=go.layout.YAxis(title=go.layout.yaxis.Title(text= y_axis,
      font=dict(family='Times New Roman', size=18, color='#000000')),
      tickmode = 'array', color="#000000", tickfont=dict(family='Times New Roman', size=18)),
      legend=dict(orientation="h", x=0.3, y=1.1,  font=dict(family='Times New Roman', size=18), bgcolor='#FFFFFF'),
        
      plot_bgcolor = '#FFFFFF')
    fig = go.Figure(bar_fig, layout=layout)
    fig.update_traces(marker_color='#C6CEF8')
    fig.add_shape(type="line", x0='P1', y0=np.mean(y_data), x1='P'+str(len(x_data)), y1=np.mean(y_data), line=dict(color='#C6CEF8', width=2, dash="dashdot"))
    fig.show()

if 'Tiles18' in loc_root_features_tiles_18:
    change_file_name = 'tiles_18_n_change.xlsx'
else:
    change_file_name = 'tiles_19_n_change.xlsx'
    
df_change = pd.read_excel(loc_root + 'Hridita/Rhythms/Findings/Jenks/'+change_file_name)
visualize_by_bar_chart(get_id_list(len(set(df_change['p_id']))), list(set(df_change['break_point'])), 'Break point')

In [ ]:
def visualize_by_box_plot(df, y_axis):
    print(np.mean(df['n_change']), np.std(df['n_change'], ddof=1))
    bar_fig = px.box(df, x="p_id", y="n_change")
    
    layout = go.Layout(
      title=go.layout.Title(text='', x=0),
      xaxis=go.layout.XAxis(title=go.layout.xaxis.Title(
      text="Participant", font=dict(family='Times New Roman', size=1, color='#000000')),
      tickmode = 'array', color="#000000", tickfont=dict(family='Times New Roman', size=1)),

      yaxis=go.layout.YAxis(title=go.layout.yaxis.Title(text= y_axis,
      font=dict(family='Times New Roman', size=18, color='#000000')),
      tickmode = 'array', color="#000000", tickfont=dict(family='Times New Roman', size=18)),
      legend=dict(orientation="h", x=0.3, y=1.1,  font=dict(family='Times New Roman', size=18), bgcolor='#FFFFFF'),
        
      plot_bgcolor = '#FFFFFF')
    fig = go.Figure(bar_fig, layout=layout)
    # fig.update_traces(marker_color='#C6CEF8')
    fig.update_xaxes(showticklabels=False)
    fig.update_layout(
    {
        "plot_bgcolor": "#FFFFFF",
        'yaxis':go.layout.YAxis(title=go.layout.yaxis.Title(text= y_axis,
      font=dict(family='Times New Roman', size=18, color='#000000')),
      tickmode = 'array', color="#000000", tickfont=dict(family='Times New Roman', size=18))
    })
    # fig.add_shape(type="line", x0='P1', y0=np.mean(y_data), x1='P195', y1=np.mean(y_data), line=dict(color='#C6CEF8', width=2, dash="dashdot"))
    fig.show()

if 'Tiles18' in loc_root_features_tiles_18:
    change_file_name = 'tiles_18_n_change.xlsx'
else:
    change_file_name = 'tiles_19_n_change.xlsx'
    
df_change = pd.read_excel(loc_root + 'Hridita/Rhythms/Findings/Jenks/'+change_file_name)
print(len(set(df_change['p_id'])))
visualize_by_box_plot(df_change, '# of change')

# Finding the data instance # for "stable" break point

In [ ]:
print(df_violin)
fig = go.Figure()
for stable_point in df_violin.columns:
    fig.add_trace(go.Violin(x=np.repeat(stable_point, df_violin.shape[0]),
                            y=df_violin[stable_point],
                            name=stable_point,
                            box_visible=True,
                            meanline_visible=True))
fig.update_layout({
        "plot_bgcolor": "#FFFFFF",
        'xaxis': go.layout.XAxis(title=go.layout.xaxis.Title(text="", font=dict(family='Times New Roman', size=18, color='#000000')),
                                 tickmode = 'array', color="#000000", tickfont=dict(family='Times New Roman', size=18)),
        'yaxis': go.layout.YAxis(title=go.layout.yaxis.Title(text="", font=dict(family='Times New Roman', size=18, color='#000000')),
                                 tickmode = 'array', color="#000000", tickfont=dict(family='Times New Roman', size=18)),
        'legend': dict(orientation="h", x=0.3, y=1.1,  font=dict(family='Times New Roman', size=1), bgcolor='#FFFFFF')})
fig.show()

# Visualizing the number of times we calculated the rhythmic parameters

In [ ]:
dic_id_n_instance = {}
for p_id in id_list:
    df_rhy_tiles18 = pd.read_excel(loc_root_features_tiles_18, p_id)
    dic_id_n_instance[p_id] = (df_rhy_tiles18.shape[0] - 1) # -1 since the first row contains NaN values (not an issue BTW)

In [ ]:
def viz_instances():
    text_font_size = 25
    tick_font_size = 20
    
    plt_data_score = go.Scatter(
        x=get_id_list(len(dic_id_n_instance.values())), y= list(dic_id_n_instance.values()),
        mode='markers',
        marker_size = list(map(lambda score: score+15, dic_id_n_instance.values())))
    
    layout = go.Layout(
        title=go.layout.Title(text='', x=0),
        xaxis=go.layout.XAxis(title=go.layout.xaxis.Title(
        text='Participants\' ID ', font=dict(family='Times New Roman', size=text_font_size, color='#000000')),
        tickmode = 'array', color="#000000", tickfont=dict(family='Times New Roman', size=tick_font_size)),
    
        yaxis=go.layout.YAxis(title=go.layout.yaxis.Title(
        text='# of instances',
        font=dict(family='Times New Roman', size=text_font_size, color='#000000')),
        tickmode = 'array', color="#000000", tickfont=dict(family='Times New Roman', size=tick_font_size)),
        plot_bgcolor = '#FFFFFF')
    
    fig = go.Figure(plt_data_score, layout=layout)
    fig.add_shape(type="line", x0='P1', y0=9, x1='P49', y1=9, line=dict(color='#000000', width=2, dash="dashdot"))
    fig
    fig.write_html("instance_score.html")

viz_instances()

# Statistical Analysis

In [ ]:
str_mean = 'mean'
str_std = 'std'
relation_data_tracker = set()
relation_list_df = []

In [ ]:
def detect_outlier_z_score(value_list):
    outliers_indices = []
    threshold = 3  # A previous study remarks to use this threshold to detect outliers
    mean = np.mean(value_list)
    std = np.std(value_list, ddof=1)

    index = 0
    for value in value_list:
        z_score = (value - mean) / std
        if np.abs(z_score) > threshold:
            outliers_indices.append(index)
        index += 1

    return outliers_indices

In [ ]:
def calculate_correlation(ds_name, depen_var, ind_var, n_missing, data_list1, data_list2):
  if len(data_list1) > 7:  # So that we can do the normaltest (N>=8 is required for this function).
      n_outlier = len(detect_outlier_z_score(data_list1))
      n_outlier += len(detect_outlier_z_score(data_list2))

      normality, p_norm_data1 = stats.normaltest(data_list1)
      normality, p_norm_data2 = stats.normaltest(data_list2)

      if p_norm_data1 > 0.05 and p_norm_data2 > 0.05 and n_outlier == 0:  # If ture, this means data are normally distributed and do not have any outlier
          cc, cc_p_value = stats.pearsonr(data_list1, data_list2)  # Pearson correlation
          r_text = "r="
      else:
          cc, cc_p_value = stats.spearmanr(data_list1, data_list2)  # Spearman correlation
          r_text = "rs="
  else:
      cc, cc_p_value = stats.spearmanr(data_list1, data_list2)  # Spearman correlation
      r_text = "rs="
  if (ds_name + depen_var + ind_var) not in relation_data_tracker:
      relation_list_df.append([ds_name, depen_var, ind_var, n_missing, r_text, cc, cc_p_value, len(data_list1)])
      relation_data_tracker.add(ds_name + depen_var + ind_var)
    
  return r_text, cc, cc_p_value, len(data_list1)

In [ ]:
def relation_with_objective_data():
    loc_root_change = 'C:/Users/wyd2hu/OneDrive - University of Virginia/Hridita/Rhythms/Findings/Jenks/'

    for sleep_var in ['deep', 'light', 'rem', 'wake', 'restless']:
        df_step = pd.DataFrame()
        df_sleep = pd.DataFrame()
    
        for loc_data in ('C:/Users/wyd2hu/S2He/AudData/Tiles18/fitbit/sleep/', 'C:/Users/wyd2hu/S2He/AudData/Tiles19/tiles-phase2-opendataset/fitbit/sleep-data/', 
                         'C:/Users/wyd2hu/S2He/AudData/Tiles18/fitbit/step-count/', 'C:/Users/wyd2hu/S2He/AudData/Tiles19/tiles-phase2-opendataset/fitbit/step-count'):
            dic_id_sleep = {}

            ind_var = ''
            depen_var = 'n_change_std'
            ds_name = ''
            n_missing_values = 0

            for p_id in os.listdir(loc_data):
                if p_id.endswith('.gz'):
                    df_temp = pd.read_csv(os.path.join(loc_data, p_id), compression='gzip')
                    df_temp.dropna(inplace=True)
                    if 'step' in loc_data:
                        ind_var = "step_" + str_mean
                        dic_id_sleep[p_id[:25]] = df_temp['StepCount'].mean()
                    else:
                        ind_var = os.path.basename(loc_data) +'_count_'+ sleep_var
                        if 'Tiles18' in loc_data:
                            # dic_id_sleep[p_id[:25]] = np.std(df_temp['duration'], ddof=1)
                            dic_id_sleep[p_id[:25]] = df_temp['sleep_phase'].tolist().count(sleep_var)
                        else:
                            # dic_id_sleep[p_id[:25]] = np.std(df_temp['seconds'], ddof=1)
                            dic_id_sleep[p_id[:25]] = df_temp['level'].tolist().count(sleep_var)

            if 'Tiles18' in loc_data:
                ds_name = 'Tiles18'
                n_change_file = 'tiles_18_n_change.xlsx'
            else:
                ds_name = 'Tiles19'
                n_change_file = 'tiles_19_n_change.xlsx'

            df_data = pd.DataFrame(dic_id_sleep.items(), columns=['p_id', 'data'])
            df_change = pd.read_excel(os.path.join(loc_root_change, n_change_file)).groupby(['p_id']).std(ddof=1).reset_index()

            print((set(df_change['p_id']) - set(df_data['p_id']))) # Participants whose data are in rhythm change, but missing in sleep, or step ...

            df_change.set_index('p_id', inplace=True)
            df_data.set_index('p_id', inplace=True)

            df_data_change = pd.merge(df_data, df_change, left_index=True, right_index=True, how='right')
            n_missing_values = df_data_change.isna().sum().sum()
            df_data_change.dropna(inplace=True)

            if "step" in loc_data:
                df_step = pd.concat([df_step, df_data_change])
            elif "sleep" in loc_data:
                df_sleep = pd.concat([df_sleep, df_data_change])

            print(n_missing_values, ds_name, depen_var, ind_var)
            print(calculate_correlation(ds_name, depen_var, ind_var, n_missing_values, df_data_change['data'], df_data_change['n_change']))

        print(calculate_correlation("both dataset", depen_var, "sleep"+sleep_var, "sum(tiles 18 & 19)", df_sleep['data'], df_sleep['n_change']))
        print(calculate_correlation("both dataset", depen_var, "step", "sum(tiles 18 & 19)", df_step['data'], df_step['n_change']))

relation_with_objective_data()

In [ ]:
def relation_with_subjective_data():
    loc_root_change = 'C:/Users/wyd2hu/OneDrive - University of Virginia/Hridita/Rhythms/Findings/Jenks/'
    df_stress = pd.DataFrame()
    
    for loc_explore_var in ("C:/Users/wyd2hu/S2He/AudData/Tiles18/surveys/scored/EMAs/stressd.csv.gz",
                           "C:/Users/wyd2hu/S2He/AudData/Tiles18/surveys/scored/EMAs/anxiety.csv.gz",
                           "C:/Users/wyd2hu/S2He/AudData/Tiles18/surveys/scored/EMAs/engage_psycap_is_cs_hs.csv.gz",
                           "C:/Users/wyd2hu/S2He/AudData/Tiles18/surveys/scored/EMAs/pand.csv.gz",
                           "C:/Users/wyd2hu/S2He/AudData/Tiles19/tiles-phase2-opendataset/surveys/scored/EMA/daily_ema.csv.gz"):
        ds_name = ''
        ind_var = ''
        depen_var = 'n_change_std'
        n_missing_values = 0

        if 'Tiles18' in loc_explore_var:
            ds_name = 'Tiles18'
        elif "Tiles19" in loc_explore_var:
            ds_name = 'Tiles19'
        
        if 'engage_psycap_is_cs_hs' in loc_explore_var:
            explore_var = 'psycap'
        elif "pand" in loc_explore_var:
            explore_var = "pand_NegAffect"
        elif "daily_ema" in loc_explore_var:
            explore_var = "stress"
        else:
            explore_var = os.path.basename(loc_explore_var).replace('.csv.gz', '')

        ind_var = explore_var + '_sum'
        df_explore_var = pd.read_csv(loc_explore_var, compression='gzip')

        if 'Tiles18' in loc_explore_var:
            df_explore_var.rename(columns={'participant_id': 'p_id', explore_var: 'explore_var'}, inplace=True)
            df_change = pd.read_excel(os.path.join(loc_root_change, 'tiles_18_n_change.xlsx')).groupby(['p_id']).std(ddof=1).reset_index()
        elif "Tiles19" in loc_explore_var:
            df_explore_var.rename(columns={'id': 'p_id', explore_var: 'explore_var'}, inplace=True)
            df_change = pd.read_excel(os.path.join(loc_root_change, 'tiles_19_n_change.xlsx')).groupby(['p_id']).std(ddof=1).reset_index()
        
        df_explore_var.dropna(inplace=True)
        df_explore_var = df_explore_var[['p_id', 'explore_var']]
        df_explore_var['p_id'] = df_explore_var['p_id'].str[:25] # we are taking the first 25 characters since we saved the rhythm info using 25 characters. The reason is the char limit in write_xlsx up to 31 char (as far as I can remember)
        df_explore_var = df_explore_var.groupby(['p_id']).sum().reset_index()
        
        print('Missing participants for which data are available in df_change, not in df_explore_var', (set(df_change['p_id']) - set(df_explore_var['p_id'])))
        
        df_change.set_index('p_id', inplace=True)
        df_explore_var.set_index('p_id', inplace=True)
        df_explore_var_change = pd.merge(df_explore_var, df_change, left_index=True, right_index=True, how='right')
        
        n_missing_values = df_explore_var_change.isna().sum().sum()
        df_explore_var_change.dropna(inplace=True)
        print(n_missing_values)

        df_explore_var_change = df_explore_var_change.astype(np.float64) 
        print(calculate_correlation(ds_name, depen_var, ind_var, n_missing_values, df_explore_var_change['explore_var'], df_explore_var_change['n_change']))

        if "stress" in ind_var:
            df_stress = pd.concat([df_stress, df_explore_var_change])
    
    print(calculate_correlation("both dataset", depen_var, 'stress', "sum(tiles 18 & 19)", df_stress['explore_var'], df_stress['n_change']))

relation_with_subjective_data()

## Save the Findings of Statistical Analysis

In [ ]:
def save_stat_findings():
    df_stat_find = pd.DataFrame(data=relation_list_df, columns = ['ds_name', 'depen_var', 'ind_var', 'n_missing',
                                                                  'r_text', 'cc', 'cc_p_value', 'len(data_list1)'])
    df_stat_find.to_excel(loc_root+"/Hridita/Rhythms/Findings/stat_findings.xlsx", index=False)
    df_stat_find.to_latex("output.tex")
save_stat_findings()

# Machine Learning

## Import Libraries

In [256]:
# !pip install boruta
# !pip install mlxtend

# !pip install hyperopt
# !pip install catboost
# !pip install scikit-optimize


from boruta import BorutaPy

import json
from lightgbm import LGBMRegressor

import joblib
import sys
sys.modules['sklearn.externals.joblib'] = joblib

import math
from mlxtend.feature_selection import SequentialFeatureSelector
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from IPython.core.display import display, HTML
import numpy as np
import os as os

import pandas as pd
from pandas.core.common import flatten
import plotly
import plotly.express as px
import plotly.graph_objects as go

import scipy.stats as stats
from sklearn import svm
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, cross_validate, LeaveOneOut, KFold, StratifiedKFold
from sklearn.linear_model import LassoCV, ElasticNetCV, RidgeCV, Lasso, ElasticNet, LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.feature_selection import mutual_info_regression
from sklearn.naive_bayes import GaussianNB

from sklearn.ensemble import GradientBoostingRegressor, VotingRegressor, RandomForestRegressor, AdaBoostRegressor, ExtraTreesRegressor, StackingRegressor

from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, scale, MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor

from sklearn.metrics import mean_squared_error, mean_absolute_error

from sklearn.feature_selection import RFE, RFECV
import xgboost as xgb
from sklearn.svm import SVR, SVC

# Installing the hyperopt library
from hyperopt import tpe, fmin, hp, STATUS_OK, Trials, space_eval
from catboost import CatBoostRegressor
from skopt.space.space import Real, Integer, Categorical
from skopt import BayesSearchCV

C:\Users\wyd2hu\AppData\Local\Temp\ipykernel_13908\14227611.py:23: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [257]:
loc_feature = 'C:/Users/wyd2hu/OneDrive - University of Virginia/Hridita/Predict/Feature/'
loc_findings = 'C:/Users/wyd2hu/OneDrive - University of Virginia/Hridita/Predict/Findings/'

rand_state = 1234
max_dep = 5

str_hyperopt = 'hyperopt_'
str_clf_name = 'clf_name'
str_params = 'params'
str_id = 'p_id'
str_data = 'data'
str_selected_features = 'Selected_Features_LOOCV'
str_length = 'Length'
str_test_id = 'Test_ID'

bool_interpret = False
str_score = 'score'

str_fpr = 'fp_'
str_tpr = 'true_pos'
str_precision = 'precision'
str_recall = 'recall'
str_curve_recall_precision = 'curve_recall_precision'
str_curve_fpr_tpr = 'curve_false_pos_true_pos'

str_inner_diff = 'inner_difference'
str_outer_diff = 'outer_difference'
str_depressed = 'Depressed'

str_dummy_clf = 'Dummy'
str_gaussian_nb='Gaussian NB'
str_light_gbm = 'Light GBM'
str_decision_tree = 'Decision Tree'
str_adaptive_boosting = 'Adaptive Boosting'
str_1_specificity = '1-Specificity'
str_stats_imp_features = 'Stats Important Features'
str_max_features = 'max_features'  # We are using max_features = n_features since we did feature selection.


# Feature Selection

## Filter Method (Info Gain)

In [234]:
def select_feature_ig(x_df, y_df, n_iteration, max_n_features):
    ig_values = mutual_info_regression(x_df, y_df, random_state=1234)
    ig_sorted_values, f_list_ig_sorted = zip(*sorted(zip(ig_values, x_df.columns.values), reverse=True))
    f_list_ig = list(f_list_ig_sorted)[:max_n_features]

    # ig_sorted_values, f_list_ig_sorted = zip(*(
    #     (ig_val, f_name) for ig_val, f_name in zip(ig_sorted_values, f_list_ig_sorted) if ig_val >= 0.1))
    # print('Ig selected features: ', len(f_list_ig_sorted), f_list_ig_sorted)
    return f_list_ig

In [235]:
def i_gain_feature_selection_and_saving(df, ds_name):
    leave_one = LeaveOneOut()
    print(loc_feature)
    print(loc_findings)

    for max_n_featues in range(5, 6): # Selecting only 5 features
      feature_data_df = []
      f_file_name = ds_name+ '_info_gain_max_feat_'+ str(max_n_featues) +'.xlsx'
      print(f_file_name)
      n_iteration = 1
        
      kf = KFold(n_splits=n_fold_for_FS, shuffle=True, random_state = rand_state)

      if os.path.exists(loc_feature) and not os.path.exists(loc_feature+f_file_name):
        for train_index, test_index in kf.split(df.drop(columns=[str_score, str_id])):
          train_x_df = df.iloc[train_index].drop(columns=[str_score, str_id])
          train_y_df = df.iloc[train_index][str_score]
          train_y_df = train_y_df.astype(int) 

          print('Max. N F.', max_n_featues, n_iteration)

          selected_features = select_feature_ig(train_x_df, train_y_df, n_iteration, max_n_featues) # Remember to change this function, if you want to use nested method
          feature_data_df.append([n_iteration, selected_features, len(selected_features)])
          n_iteration += 1

        feature_df = pd.DataFrame(data=feature_data_df, columns=[str_test_id, str_selected_features, str_length])
        feature_df.to_excel(loc_feature + f_file_name, index=False)


## Embedded Method (Random Forest)

In [236]:
def select_feature_rand_forest(x_df, y_df, max_n_features):
    rf_clf_fea = RandomForestRegressor(random_state= rand_state,
                                        max_depth= max_dep)
    rf_clf_fea.fit(x_df, y_df)
    imp_val_list = rf_clf_fea.feature_importances_
    imp_val_sorted_values, fea_list = zip(*sorted(zip(imp_val_list, x_df.columns.values), reverse=True))
    fea_list = list(fea_list)[:max_n_features]

    return fea_list

In [237]:
def rand_forest_feature_selection_and_saving(df, ds_name):
    leave_one = LeaveOneOut()
    print(loc_feature)
    print(loc_findings)

    for max_n_featues in range(5, 6): # just as a test, we are keeping max 5 features
      feature_data_df = []
      f_file_name = ds_name + '_rand_forest_max_feat_'+ str(max_n_featues) +'.xlsx'
      print(f_file_name)
      n_iteration = 1

      kf = KFold(n_splits=n_fold_for_FS, shuffle=True, random_state = rand_state)

      if os.path.exists(loc_feature) and not os.path.exists(loc_feature+f_file_name):
        for train_index, test_index in kf.split(df.drop(columns=[str_score, str_id])):
          train_x_df = df.iloc[train_index].drop(columns=[str_score, str_id])
          train_y_df = df.iloc[train_index][str_score]
          train_y_df = train_y_df.astype(int) 

          print('Max. N Feature', max_n_featues, n_iteration)

          selected_features = select_feature_rand_forest(train_x_df, train_y_df, max_n_featues) # Remember to change this function, if you want to use nested method
          feature_data_df.append([n_iteration, selected_features, len(selected_features)])
          n_iteration += 1

        feature_df = pd.DataFrame(data=feature_data_df, columns=[str_test_id, str_selected_features, str_length])
        feature_df.to_excel(loc_feature + f_file_name, index=False)

## Bat for Feature Selection (Used a Bit Modified Code of Online for This Case)

In [248]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


# load data
# data  = pd.read_csv('ionosphere.csv')
# data  = data.values
# feat  = np.asarray(data[:, 0:-1])
# label = np.asarray(data[:, -1])

# print(feat)
# print(label)

# split data into train & validation (70 -- 30)

def _by_bat(train_x, train_y):
    loc_root_model_data =  'C:/Users/wyd2hu/OneDrive - University of Virginia/Hridita/Predict/Data/'
    
    feat  = np.asarray(train_x.values)
    label = np.asarray(train_y)

    xtrain, xtest, ytrain, ytest = train_test_split(feat, label, test_size=0.3, random_state=1234)
    fold = {'xt':xtrain, 'yt':ytrain, 'xv':xtest, 'yv':ytest}

    # parameter
    k    = 5     # k-value in KNN
    N    = 10    # number of chromosomes
    T    = 100   # maximum number of generations
    CR   = 0.8
    MR   = 0.01
    opts = {'k':k, 'fold':fold, 'N':N, 'T':T, 'CR':CR, 'MR':MR}

    # perform feature selection
    fmdl = jfs(feat, label, opts)
    sf   = fmdl['sf']

    return  list(df_data.drop(columns=['p_id']).columns[sf])

    # # model with selected features
    # num_train = np.size(xtrain, 0)
    # num_valid = np.size(xtest, 0)
    # x_train   = xtrain[:, sf]
    # y_train   = ytrain.reshape(num_train)  # Solve bug
    # x_valid   = xtest[:, sf]
    # y_valid   = ytest.reshape(num_valid)  # Solve bug

    # mdl       = KNeighborsClassifier(n_neighbors = k)
    # mdl.fit(x_train, y_train)

    # # accuracy
    # y_pred    = mdl.predict(x_valid)
    # Acc       = np.sum(y_valid == y_pred)  / num_valid
    # print("Accuracy:", 100 * Acc)

    # # number of selected features
    # num_feat = fmdl['nf']
    # print("Feature Size:", num_feat)

    # # plot convergence
    # curve   = fmdl['c']
    # curve   = curve.reshape(np.size(curve,1))
    # x       = np.arange(0, opts['T'], 1.0) + 1.0

    # fig, ax = plt.subplots()
    # ax.plot(x, curve, 'o-')
    # ax.set_xlabel('Number of Iterations')
    # ax.set_ylabel('Fitness')
    # ax.set_title('GA')
    # ax.grid()
    # plt.show()

In [249]:
def bat_alg_feature_selection_and_saving(df, ds_name):
  leave_one = LeaveOneOut()
  print(loc_feature)
  print(loc_findings)

  feature_data_df = []
  f_file_name = ds_name+'_bat_alg_.xlsx'
  print(f_file_name)
  n_iteration = 1

  kf = KFold(n_splits=n_fold_for_FS, shuffle=True, random_state = rand_state)

  if os.path.exists(loc_feature) and not os.path.exists(loc_feature+f_file_name):
    for train_index, test_index in kf.split(df.drop(columns=[str_score, str_id])):
      train_x_df = df.iloc[train_index].drop(columns=[str_score, str_id])
      train_y_df = df.iloc[train_index][str_score]
      train_y_df = train_y_df.astype(int) 

      selected_features = _by_bat(train_x_df, train_y_df) # Remember to change this function, if you want to use nested method
      feature_data_df.append([n_iteration, selected_features, len(selected_features)])
      n_iteration += 1

    feature_df = pd.DataFrame(data=feature_data_df, columns=[str_test_id, str_selected_features, str_length])
    feature_df.to_excel(loc_feature + f_file_name, index=False)

## Save Features by Using Different Algo

In [254]:
n_fold_for_FS = 5

def select_feature():
    loc_root_model_data =  'C:/Users/wyd2hu/OneDrive - University of Virginia/Hridita/Predict/Data/'
    for file_name in os.listdir(loc_root_model_data):
        df_data = pd.read_excel(os.path.join(loc_root_model_data, file_name))
        df_data.rename(columns={'n_change': str_score}, inplace=True)
        ds_name = file_name.replace('.xlsx', '')
        
        # i_gain_feature_selection_and_saving(df_data, ds_name)
        # rand_forest_feature_selection_and_saving(df_data, ds_name)
        bat_alg_feature_selection_and_saving(df_data, ds_name)
select_feature()

C:/Users/wyd2hu/OneDrive - University of Virginia/Hridita/Predict/Feature/
C:/Users/wyd2hu/OneDrive - University of Virginia/Hridita/Predict/Findings/
Tiles18feature_lebel_bat_alg_.xlsx
Generation: 1
Best (BA): 9.842779331306991
Generation: 1
Best (BA): 11.520632218844987
Generation: 1
Best (BA): 16.163998176291795
Generation: 1
Best (BA): 17.724607902735563
Generation: 1
Best (BA): 10.020301519756837
C:/Users/wyd2hu/OneDrive - University of Virginia/Hridita/Predict/Feature/
C:/Users/wyd2hu/OneDrive - University of Virginia/Hridita/Predict/Findings/
Tiles19feature_lebel_bat_alg_.xlsx
Generation: 1
Best (BA): 0.6591714285714285
Generation: 1
Best (BA): 0.7500285714285714
Generation: 1
Best (BA): 0.5308571428571429
Generation: 1
Best (BA): 0.47877142857142857
Generation: 1
Best (BA): 0.2734


# `Hyper-parameter Tuning`

## `Bayesian Optimization Technique`

In [242]:
def hyperparameter_tuning(params):
  global crt_clf, x_data, y_data  # Remember, these are the training data
  global ensemble_Regressor_list, hyperopt_crt_split

  clf = crt_clf(**params)
  skf = StratifiedKFold(n_splits=hyperopt_crt_split)
  rmse = cross_val_score(clf, x_data, y_data.values.tolist(), scoring="neg_root_mean_squared_error", cv=skf).mean()
  return {"loss": -rmse, "status": STATUS_OK}

In [259]:
def bayesian_search(model_name, model, m_parameters, df):
    global crt_clf, x_data, y_data, test_id_feature_dic, dic_model_balanced_acc, dic_model_auc
    global df_threshold, df_fpr_tpr, df_prec_recall_rate
    global crt_str_ensemble_model, ensemble_Regressor_list, hyperopt_crt_split
    global shap_train_df, shap_test_df, crt_interpret_m
    print(model_name)

    kf = LeaveOneOut()
    actual_scores = []
    predicted_scores = []
    n_iteration = 1
    com_imp_features_set = set()
    shap_test_df = pd.DataFrame()
    shap_train_df = pd.DataFrame()
    print(opt_library)

    kf = KFold(n_splits=n_fold_for_FS, shuffle=True, random_state = rand_state)
    list_all_id = list(range(1, n_fold_for_FS+1))
    
    for train_index, test_index in kf.split(df.drop(columns=[str_score, str_id])):
      important_features = test_id_feature_dic[n_iteration]
      com_imp_features_set.update(important_features)

      ##### Control Center #### After feature selection
      train_x_df = df.iloc[train_index][important_features]
      train_y_df = df.iloc[train_index][str_score]
      test_x_df = df.iloc[test_index][important_features]
      test_y_df = df.iloc[test_index][str_score]

      # The following data are for hyper-parameter tuning. Do not remove. Search at first, if you want to remove.
      if str_max_features in m_parameters:
        m_parameters[str_max_features] = hp.choice(str_max_features, [len(important_features)])

      if opt_library == str_hyperopt and model_name != str_gaussian_nb and model_name != str_dummy_clf:
        hyperopt_crt_split = 3
        crt_clf = model
        x_data = train_x_df.copy()
        y_data = train_y_df.copy()
        gbm_best_param = fmin(fn=hyperparameter_tuning,
                              space=m_parameters, max_evals=5,
                              rstate=np.random.default_rng(rand_state),
                              algo=tpe.suggest)
        gbm_best_param = space_eval(m_parameters, gbm_best_param)
        best_model = model(**gbm_best_param)
        best_model.fit(train_x_df, train_y_df.tolist())

      elif opt_library == str_skopt:
        if crt_str_ensemble_model == str_weighted_vot_clf:
          weights_ensemble_list = get_weights_ensemble_clf(train_x_df, train_y_df)

          crt_clf = model(estimators=ensemble_Regressor_list, weights=weights_ensemble_list, voting='soft')
          x_data = train_x_df.copy()
          y_data = train_y_df.copy()
          bayes_skopt_model = BayesSearchCV(crt_clf, m_parameters, n_iter=50, n_jobs=-1,
                                            cv=StratifiedKFold(n_splits=20), scoring='f1',
                                            random_state = rand_state)
          fitted_model = bayes_skopt_model.fit(train_x_df, train_y_df.tolist())
          best_model = fitted_model.best_estimator_
        elif crt_str_ensemble_model == str_stack_clf:
          crt_clf = model
          x_data = train_x_df.copy()
          y_data = train_y_df.copy()
          crt_clf = model(estimators = ensemble_Regressor_list,
                          final_estimator=LogisticRegression(random_state=rand_state, dual=True,
                                                             penalty='l2', solver='liblinear', max_iter=15000,
                                                             class_weight='balanced'),
                          cv=StratifiedKFold(n_splits=10), stack_method = 'predict_proba')
          bayes_skopt_model = BayesSearchCV(crt_clf, m_parameters, n_iter=50, n_jobs=-1,
                                            cv=StratifiedKFold(n_splits=20), scoring='f1',
                                            random_state = rand_state)
          fitted_model = bayes_skopt_model.fit(train_x_df.values, train_y_df.tolist())
          best_model = fitted_model.best_estimator_

      if model_name == str_gaussian_nb:
        best_model = model()
        best_model.fit(train_x_df.values, train_y_df.tolist())
      elif model_name == str_dummy_clf:
        best_model = model
        best_model.fit(train_x_df.values, train_y_df.tolist())

      actual_scores.extend(test_y_df.values.tolist())
      predicted_scores.extend(best_model.predict(test_x_df.values))

      if bool_interpret:
        ## Interpreting the model - ############### start
        if crt_interpret_m == str_explainer:
          m_explainer = shap.Explainer(best_model)
        elif crt_interpret_m == str_kernel_explainer:
          m_explainer = shap.KernelExplainer(best_model.predict_proba, train_x_df.values)

        print(crt_interpret_m, m_explainer)
        shap_test_val = m_explainer.shap_values(test_x_df)
        shap_train_val = m_explainer.shap_values(train_x_df)

        shap_test_single_df = pd.DataFrame(data = shap_test_val[1], columns=test_x_df.columns)
        shap_test_single_df.index=[test_id]

        shap_train_sit_df = pd.DataFrame(data = shap_train_val[1], columns=train_x_df.columns) # sit: single iteration
        shap_train_sit_df.index = train_id_list

        if shap_test_df.shape[0] == 0:
          shap_test_df = shap_test_single_df.copy()
          shap_train_df = shap_train_sit_df.copy()
        else:
          shap_test_df = pd.concat([shap_test_df, shap_test_single_df], axis=0, join='inner')
          shap_train_df = pd.concat([shap_train_df, shap_train_sit_df], axis=0, join='inner')
          # shap_train_df = shap_train_df.groupby(shap_train_df.index).mean() # Uncomment, if you wish to have a participant's data for single time

        # Force Plot
        local_visualize_shap_values(m_explainer, shap_test_val, test_x_df, 'P_'+str(n_iteration)+'_ID_'+
                                    test_id+ '_Actual_'+ str(test_y_df.values.tolist()) + '_Predict_'+
                                    str(best_model.predict(test_x_df.values))+
                                    os.path.basename(feature_loc).replace('.xlsx', ''))
        # Interpreting the model- ############### End

      print(n_iteration, test_y_df.values.tolist(), best_model.predict(test_x_df.values))
      # print(str(fitted_model.best_params_), '\n\n', test_y_df.values.tolist(), best_model.predict(test_x_df), '\n', important_features)
      n_iteration += 1
    
      rmse = mean_squared_error(actual_scores, predicted_scores)
      mae = mean_absolute_error(actual_scores, predicted_scores)

#     true_neg, false_pos, false_neg, true_pos = confusion_matrix(actual_scores, predicted_scores).ravel()
#     specificity = true_neg / (true_neg+false_pos)

#     flase_pr, true_pr, thresholds_roc = roc_curve(actual_scores, predicted_scores, pos_label=1)
#     auc_score = auc(flase_pr, true_pr)
#     precision_rate, recall_rate, thresholds_pre_rec = precision_recall_curve(actual_scores, predicted_scores, pos_label=1)

#     sensitivity = recall_score(actual_scores, predicted_scores)
#     precision = precision_score(actual_scores, predicted_scores)
#     f_1score = f1_score(actual_scores, predicted_scores)
#     bal_accuracy = balanced_accuracy_score(actual_scores, predicted_scores)
#     accuracy = accuracy_score(actual_scores, predicted_scores)

#     dic_model_balanced_acc[model_name] = bal_accuracy
#     dic_model_auc[model_name] = auc_score

#     if df_fpr_tpr.shape[0] == 0 or df_fpr_tpr.shape[0] == len(flase_pr):
#         df_fpr_tpr[model_name + str_fpr] = flase_pr
#         df_fpr_tpr[model_name + str_tpr] = true_pr
#         df_fpr_tpr[model_name + str_threshold] = thresholds_roc
#         print('ROC', thresholds_roc)
#         print('Precision Recall', thresholds_pre_rec)

#         df_prec_recall_rate[model_name + str_precision] = precision_rate
#         df_prec_recall_rate[model_name + str_recall] = recall_rate
#         df_threshold[model_name + str_curve_recall_precision + str_threshold] = thresholds_pre_rec
#     else:
#         print('\n\n\nSorry,Failed to assign '+model_name+ 's false positive rate and true positive rate values')

#     print('\n\n\n F1 Score', f_1score)
#     print('Sensitivity', sensitivity)
#     print('Specificity', specificity)
#     print('Precision', precision)
#     print('Accuracy', accuracy_score(actual_scores, predicted_scores))
#     print('Balanced Accuracy', bal_accuracy)
#     print('AUC', auc_score)
#     # print('Common Important Feature Set ', len(com_imp_features_set), com_imp_features_set)

    classif_result_list_df.append([model_name, rmse, mae])
    print(*classif_result_list_df, sep='\n')
    return best_model

# `Model Development`

In [260]:
def get_features():
  global test_id_feature_dic, feature_loc, FLAG_NOT_0_Features
  test_id_feature_dic = {}
  df_imp_features = pd.read_excel(feature_loc)
  FLAG_NOT_0_Features = True

  for test_id, single_LOOCV_f_list in zip(df_imp_features[str_test_id].values, df_imp_features[str_selected_features].values):
    features_in_a_iter = json.loads(single_LOOCV_f_list.replace('\'', '"').replace('{', '[').replace('}', ']'))  # Some lists of data are set as set ('{', '}')
    if len(features_in_a_iter) > 0:
      test_id_feature_dic[test_id] = features_in_a_iter
    else:
      FLAG_NOT_0_Features = False
      break

In [261]:
def add_clf(clf_name, Regressor, clf_params, bool_individual_model, dic_clf_details, df):
  dic_clf_details[clf_name.replace(' ', '')] = {}
  dic_clf_details[clf_name.replace(' ', '')][str_clf_name] = Regressor
  dic_clf_details[clf_name.replace(' ', '')][str_params] = clf_params

  if bool_individual_model:
    bayesian_search(clf_name, Regressor, clf_params, df)

## Hyperopt

In [262]:
def hyperopt_classify(df):
  global rand_state, feature_loc, dic_model_balanced_acc, dic_model_auc, hyperopt_dic_clf_details
  global max_dep, df_threshold, df_fpr_tpr, df_prec_recall_rate, classif_result_list_df
  global FLAG_NOT_0_Features, opt_library
  column_list = ['Model Name', 'rmse', 'mae']

  if os.path.exists(loc_findings):
    individual_model = True
    if individual_model:
      f_file_list_for_ml = sorted(os.listdir(loc_feature))
    else:
      f_file_list_for_ml = ['DummyLoc.xlsx']
    print('File List', *f_file_list_for_ml, sep='\n')

    for f_file in f_file_list_for_ml: # f: feature
      h_library = str_hyperopt # Must be Hyperopt
      opt_library = h_library
      print('Working\n\n', f_file)

      if f_file.endswith('.xlsx') and not os.path.exists(loc_findings +'//'+ h_library+f_file):
        feature_loc = loc_feature +'//'+ f_file

        if individual_model:
          get_features()
        else:
          FLAG_NOT_0_Features = True

        if FLAG_NOT_0_Features:
          ### Classification
          classif_result_list_df = []

          dic_score_metrics = {}
          dic_model_balanced_acc = {}
          dic_model_auc = {}
          hyperopt_dic_clf_details = {}
          dic_method_id_feature_list = {}  # Feature Selection Method - LOOCV TEST ID - Feature Set
          dic_method_id_feature_list[os.path.basename(f_file)] = {}

          df_fpr_tpr = pd.DataFrame()
          df_prec_recall_rate = pd.DataFrame()
          df_threshold = pd.DataFrame()

          n_estimators = 20

          #### KNN Regressor
          knn_clf = KNeighborsRegressor
          knn_params = {'n_neighbors': hp.choice('n_neighbors', range(2, 5))}
          add_clf('KNN', knn_clf, knn_params, individual_model, hyperopt_dic_clf_details, df)

#           #### MLPRegressor
          # mlp_clf = MLPRegressor
          # mlp_params = {'hidden_layer_sizes': hp.choice('hidden_layer_sizes', range(1, 20)),
          #               'max_iter': hp.choice('max_iter', range(1, 75000, 100)),
          #               'learning_rate': hp.choice('learning_rate', ['invscaling', 'adaptive']),
          #               'random_state': rand_state}
          # add_clf('MLPRegressor', mlp_clf, mlp_params, individual_model, hyperopt_dic_clf_details, df)

          ### Decision Tree Regressor
          dt_clf = DecisionTreeRegressor
          dt_params = {'max_depth': hp.choice('max_depth', range(2, max_dep)),
                      'random_state': rand_state}
          add_clf('Decision Tree', dt_clf , dt_params, individual_model, hyperopt_dic_clf_details, df_data)

          ### Random Forest Regressor
          random_forest_clf = RandomForestRegressor
          rf_params = {'n_estimators': hp.choice('n_estimators', range(2, n_estimators)),
                      'max_depth': hp.choice('max_depth', range(2, max_dep)),
                      'random_state': rand_state}
          add_clf('Random Forest' , random_forest_clf, rf_params, individual_model, hyperopt_dic_clf_details, df_data)

          #### CatBoostRegressor
          cat_clf = CatBoostRegressor
          cat_params = {'iterations': hp.choice('iterations', range(2, n_estimators)),
                        'depth': hp.choice('depth', range(2, max_dep)),
                        'learning_rate': hp.uniform('learning_rate', 0.01, 1),
                        'logging_level': 'Silent',
                        'random_seed': rand_state}
          add_clf('CatBoostRegressor', cat_clf, cat_params, individual_model, hyperopt_dic_clf_details, df_data)

#           #### Adaptive Boosting Regressor
          ada_boost_clf = AdaBoostRegressor
          ada_clf_params = {'n_estimators': hp.choice('n_estimators', range(2, n_estimators)),
                            'base_estimator': DecisionTreeRegressor(max_depth=max_dep),
                            'learning_rate': hp.uniform('learning_rate', 0.01, 1),
                            'random_state': rand_state}
          add_clf('Adaptive Boosting', ada_boost_clf, ada_clf_params, individual_model, hyperopt_dic_clf_details, df_data)

#           #### Gradient Boosting Regressor
          gb_clf = GradientBoostingRegressor
          gbr_params =  {'n_estimators': hp.choice('n_estimators', range(2, n_estimators)),
                        'max_depth': hp.choice('max_depth', range(2, max_dep)),
                        'learning_rate': hp.uniform('learning_rate', 0.01, 1),
                        'random_state': rand_state}
          add_clf('Gradient Boosting', gb_clf, gbr_params, individual_model, hyperopt_dic_clf_details, df_data)

#           #### ExtraTrees Regressor
          extra_tree_clf = ExtraTreesRegressor
          extra_tree_params = {'n_estimators': hp.choice('n_estimators', range(2, n_estimators)),
                              'max_depth': hp.choice('max_depth', range(2, max_dep)),
                              'random_state': rand_state}
          add_clf('Extra Tree Regressor', extra_tree_clf, extra_tree_params, individual_model, hyperopt_dic_clf_details, df_data)

#           #### Extreme Gradient Boosting Regressor
          xgb_clf = xgb.XGBRegressor
          xgb_params = {'learning_rate': hp.uniform('learning_rate', 0.01, 1),
                        'n_estimators': hp.choice('n_estimators', range(2, n_estimators)),
                        'max_depth': hp.choice('max_depth', range(2, max_dep)),
                        'random_state': rand_state}
          add_clf('XGBoosting', xgb_clf, xgb_params, individual_model, hyperopt_dic_clf_details, df_data)

          if individual_model:
            df_classific_performance = pd.DataFrame(data = classif_result_list_df,
                                                    columns=column_list)
            df_classific_performance.to_excel(loc_findings +'//'+ h_library +f_file, index=False)

#             if not os.path.exists(loc_findings +'//'+ h_library +f_file):
#               h_library = 'hyperopt_'
#             df_classific_performance = pd.read_excel(loc_findings +'//'+ h_library +f_file)
#             df_classific_performance = df_classific_performance.append(gaus_perf_dic, ignore_index=True)
#             print(gaus_perf_dic)
#             print(df_classific_performance)
#             df_classific_performance.to_excel(loc_findings +'//'+ h_library +f_file, index=False)

#             print('Done ', f_file)

loc_root_model_data =  'C:/Users/wyd2hu/OneDrive - University of Virginia/Hridita/Predict/Data/'
for file_name in os.listdir(loc_root_model_data):
    df_data = pd.read_excel(os.path.join(loc_root_model_data, file_name))
    df_data.rename(columns={'n_change': str_score}, inplace=True)
    hyperopt_classify(df_data)

File List
Tiles18feature_lebel_bat_alg_.xlsx
Tiles18feature_lebel_info_gain_max_feat_5.xlsx
Tiles18feature_lebel_rand_forest_max_feat_5.xlsx
Tiles19feature_lebel_bat_alg_.xlsx
Tiles19feature_lebel_info_gain_max_feat_5.xlsx
Tiles19feature_lebel_rand_forest_max_feat_5.xlsx
Working

 Tiles18feature_lebel_bat_alg_.xlsx
KNN
hyperopt_
100%|█████████████████████████████████████████████████| 5/5 [00:00<00:00, 106.62trial/s, best loss: 3.4118933939885863]
1 [21, 16, 20, 19, 14, 13, 19, 14, 18, 19, 20, 19, 19, 9, 20, 18, 13, 22, 23, 21, 22, 19, 22, 17, 19, 20, 21, 22, 21, 11, 19, 21, 21, 23, 18, 18, 21, 18, 15] [15.5  17.75 19.25 20.   21.   19.5  18.25 19.75 12.25 16.75 20.25 22.
 16.   20.   19.5  19.5  20.   18.   19.   18.75 19.25 20.25 21.   16.5
 19.25 15.5  19.5  19.25 17.75 21.5  20.25 15.25 19.   18.25 19.5  20.25
 21.25 19.5  12.25]
100%|█████████████████████████████████████████████████| 5/5 [00:00<00:00, 103.31trial/s, best loss: 3.4297819442579147]
2 [18, 20, 18, 11, 23, 20, 20, 18, 

C:\Users\wyd2hu\AppData\Roaming\Python\Python311\site-packages\sklearn\model_selection\_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(

C:\Users\wyd2hu\AppData\Roaming\Python\Python311\site-packages\sklearn\model_selection\_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(

C:\Users\wyd2hu\AppData\Roaming\Python\Python311\site-packages\sklearn\model_selection\_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(

C:\Users\wyd2hu\AppData\Roaming\Python\Python311\site-packages\sklearn\model_selection\_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(

C:\Users\wyd2hu\AppData\Roaming\Python\Python311\site-packages\sklearn\model_selection\_split.py:684: UserWarning: The least populated class

100%|█████████████████████████████████████████████████| 5/5 [00:00<00:00, 128.84trial/s, best loss: 3.4674807597630966]
4 [22, 22, 17, 22, 22, 19, 9, 12, 21, 18, 16, 17, 19, 19, 23, 16, 22, 22, 21, 21, 23, 20, 16, 21, 18, 16, 17, 23, 20, 14, 20, 19, 12, 12, 20, 21, 17, 22] [19.   19.   17.75 19.5  19.25 19.5  20.25 20.25 18.5  18.5  19.5  19.5
 16.5  17.25 20.75 18.25 18.25 18.75 19.75 18.5  20.   16.5  19.5  20.5
 18.75 18.25 14.   20.   20.   20.   18.25 18.5  15.75 20.25 19.5  19.75
 19.   17.  ]
100%|██████████████████████████████████████████████████| 5/5 [00:00<00:00, 159.87trial/s, best loss: 3.659469611990295]
5 [19, 19, 17, 16, 22, 18, 17, 19, 19, 20, 20, 8, 22, 15, 23, 19, 21, 19, 19, 14, 21, 21, 17, 17, 13, 21, 19, 23, 19, 22, 19, 16, 21, 23, 18, 18, 18, 20] [20.25 21.5  21.5  19.   18.25 20.75 19.   19.75 21.25 21.   21.25 19.5
 18.75 18.   18.25 20.75 20.5  19.5  19.5  20.75 19.   21.75 17.75 19.5
 19.25 18.75 19.   14.5  20.75 21.   20.5  20.   16.5  18.   17.25 18.75
 18.

C:\Users\wyd2hu\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(
C:\Users\wyd2hu\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(
C:\Users\wyd2hu\AppData\Roaming\Python\Python311\site-packages\sklearn\model_selection\_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(

C:\Users\wyd2hu\AppData\Roaming\Python\Python311\site-packages\sklearn\model_selection\_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(

C:\Users\wyd2hu\AppData\Roaming\Python\Python311\site-packages\sklearn\model_selection\_split.py:684: UserWarning: The least populated class in y has only 1 mem

100%|█████████████████████████████████████████████████| 5/5 [00:00<00:00, 157.79trial/s, best loss: 3.3935762933940055]
4 [22, 22, 17, 22, 22, 19, 9, 12, 21, 18, 16, 17, 19, 19, 23, 16, 22, 22, 21, 21, 23, 20, 16, 21, 18, 16, 17, 23, 20, 14, 20, 19, 12, 12, 20, 21, 17, 22] [19.14942529 19.14942529 19.14942529 18.46153846 19.14942529 19.14942529
 20.44       20.44       19.14942529 18.46153846 19.14942529 19.14942529
 18.46153846 18.46153846 17.68       19.14942529 19.14942529 17.68
 19.14942529 19.14942529 19.14942529 19.14942529 19.14942529 19.14942529
 20.44       18.46153846 19.14942529 19.14942529 19.14942529 19.14942529
 19.14942529 19.14942529  7.5        20.44       19.14942529 19.14942529
 19.14942529 18.46153846]
100%|█████████████████████████████████████████████████| 5/5 [00:00<00:00, 159.90trial/s, best loss: 3.5624270206485256]
5 [19, 19, 17, 16, 22, 18, 17, 19, 19, 20, 20, 8, 22, 15, 23, 19, 21, 19, 19, 14, 21, 21, 17, 17, 13, 21, 19, 23, 19, 22, 19, 16, 21, 23, 18, 18, 18

...

100%|███████████████████████████████████████████████████| 5/5 [00:00<00:00, 22.78trial/s, best loss: 3.378096975005049]
3 [17, 22, 19, 20, 19, 21, 20, 17, 22, 18, 21, 22, 18, 17, 17, 22, 18, 21, 22, 20, 19, 18, 18, 22, 15, 22, 21, 10, 22, 20, 20, 21, 19, 9, 13, 18, 17, 23, 22] [18.739683 19.065456 18.570501 18.213793 18.875296 19.065456 18.756725
 18.875296 18.875296 18.875296 17.700518 18.54696  18.570501 18.70939
 19.141699 18.820868 18.0205   18.34604  18.174337 18.540255 19.065456
 18.70939  19.065456 18.875296 18.875296 18.570501 18.875296 18.958908
 18.875296 18.570501 19.065456 18.891333 18.70939  18.540255 18.540255
 18.70939  18.875296 18.54696  19.065456]
100%|███████████████████████████████████████████████████| 5/5 [00:00<00:00, 35.46trial/s, best loss: 3.268170743742113]
4 [22, 22, 17, 22, 22, 19, 9, 12, 21, 18, 16, 17, 19, 19, 23, 16, 22, 22, 21, 21, 23, 20, 16, 21, 18, 16, 17, 23, 20, 14, 20, 19, 12, 12, 20, 21, 17, 22] [19.005772 19.005772 19.220526 18.732456 19.005772 1

C:\Users\wyd2hu\AppData\Roaming\Python\Python311\site-packages\sklearn\model_selection\_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(

C:\Users\wyd2hu\AppData\Roaming\Python\Python311\site-packages\sklearn\model_selection\_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(

C:\Users\wyd2hu\AppData\Roaming\Python\Python311\site-packages\sklearn\model_selection\_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(

C:\Users\wyd2hu\AppData\Roaming\Python\Python311\site-packages\sklearn\model_selection\_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(

C:\Users\wyd2hu\AppData\Roaming\Python\Python311\site-packages\sklearn\model_selection\_split.py:684: UserWarning: The least populated class

100%|███████████████████████████████████████████████████| 5/5 [00:00<00:00, 22.78trial/s, best loss: 3.389374734670282]

C:\Users\wyd2hu\AppData\Roaming\Python\Python311\site-packages\sklearn\model_selection\_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(

C:\Users\wyd2hu\AppData\Roaming\Python\Python311\site-packages\sklearn\model_selection\_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(

C:\Users\wyd2hu\AppData\Roaming\Python\Python311\site-packages\sklearn\model_selection\_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(

C:\Users\wyd2hu\AppData\Roaming\Python\Python311\site-packages\sklearn\model_selection\_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(




5 [19, 19, 17, 16, 22, 18, 17, 19, 19, 20, 20, 8, 22, 15, 23, 19, 21, 19, 19, 14, 21, 21, 17, 17, 13, 21, 19, 23, 19, 22, 19, 16, 21, 23, 18, 18, 18, 20] [17.47975  18.977383 18.738222 19.470936 18.738222 18.977383 18.738222
 18.738222 18.738222 18.977383 18.738222 18.738222 18.977383 18.977383
 18.977383 18.412025 18.977383 18.738222 19.228756 18.738222 18.738222
 18.977383 18.977383 18.977383 19.470936 18.977383 19.230356 17.281507
 18.977383 18.977383 18.738222 18.977383 18.977383 19.230356 18.412025
 17.281507 18.738222 18.977383]
['KNN', 14.106325561312609, 2.8700345423143356]
['Decision Tree', 13.360690837764523, 2.687106853922371]
['Random Forest', 11.198069425026697, 2.4890803254295157]
['CatBoostRegressor', 11.578939259032687, 2.5503683035970908]
['Adaptive Boosting', 13.185996849157837, 2.6760147780719854]
['Gradient Boosting', 11.167986714232411, 2.508951598521844]
['Extra Tree Regressor', 11.371583037286358, 2.516163389599401]
['XGBoosting', 11.273125996039212, 2.495661661

# Extracting Features for ML

In [ ]:
def add_data_to_df(data_list):
    single_data_list = []
    
    single_data_list.append(min(data_list))
    single_data_list.append(max(data_list))
    single_data_list.append(np.mean(data_list))
    single_data_list.append(np.std(data_list, ddof=1))
    single_data_list.append(stats.iqr(data_list, nan_policy='raise'))
    single_data_list.append(stats.skew(data_list, bias=False))
    single_data_list.append(stats.kurtosis(data_list, bias=False))
    
    return single_data_list

In [ ]:
print(os.path.basename('C:/Users/wyd2hu/S2He/AudData/Tiles18/fitbit/sleep'))

In [19]:
def extract_features():
    loc_root_change = 'C:/Users/wyd2hu/OneDrive - University of Virginia/Hridita/Rhythms/Findings/Jenks/'
    
    for list_loc_data in (['C:/Users/wyd2hu/S2He/AudData/Tiles18/fitbit/sleep', 'C:/Users/wyd2hu/S2He/AudData/Tiles18/fitbit/step-count'], 
                          ['C:/Users/wyd2hu/S2He/AudData/Tiles19/tiles-phase2-opendataset/fitbit/sleep-data', 'C:/Users/wyd2hu/S2He/AudData/Tiles19/tiles-phase2-opendataset/fitbit/step-count']):        
        df_data = pd.DataFrame({'p_id':[]})
        df_data.set_index('p_id', inplace=True)
        
        for loc_data in list_loc_data:
            
            data_features_list = []
            
            for p_id in os.listdir(loc_data):
                single_data_list = [p_id[:25]]
                feature_name_list = []
                feature = ''
                
                if p_id.endswith('.gz'):
                    df_temp = pd.read_csv(os.path.join(loc_data, p_id), compression='gzip')
                    df_temp.dropna(inplace=True)

                    if 'step' in loc_data:
                        feature = 'step'
                        single_data_list.extend(add_data_to_df(df_temp['StepCount'].tolist()))
                    elif 'sleep' in loc_data:
                        feature = 'sleep'
                        if 'Tiles18' in loc_data:
                            single_data_list.extend(add_data_to_df(df_temp['duration'].tolist()))
                        else:
                            if len(df_temp['seconds'].tolist()) > 0:
                                single_data_list.extend(add_data_to_df(df_temp['seconds'].tolist()))
                
                feature_name_list.append(feature + '_min')
                feature_name_list.append(feature + '_max')
                feature_name_list.append(feature + '_mean')
                feature_name_list.append(feature + '_STD')
                feature_name_list.append(feature + '_IQR')
                feature_name_list.append(feature + '_skew')
                feature_name_list.append(feature + '_kurtosis')
            
                if (len(single_data_list)-1) == len(feature_name_list): # -1 for p_id. to avoid the cases where there was empty file
                    data_features_list.append(single_data_list)
            
            list_column = ['p_id'] 
            list_column.extend(feature_name_list)
            df_single_data = pd.DataFrame(data = data_features_list, columns = list_column)
            df_single_data.set_index('p_id', inplace=True)
            
            df_data = pd.merge(df_data, df_single_data, left_index=True, right_index=True, how='right')
            
        if 'Tiles18' in loc_data:
            ds_name = 'Tiles18'
            n_change_file = 'tiles_18_n_change.xlsx'
        else:
            ds_name = 'Tiles19'
            n_change_file = 'tiles_19_n_change.xlsx'

        df_change = pd.read_excel(os.path.join(loc_root_change, n_change_file)).groupby(['p_id']).sum().reset_index()
        df_change = df_change[['p_id', 'n_change']]
        df_change.set_index('p_id', inplace=True)
        
        df_data_change = pd.merge(df_data, df_change, left_index=True, right_index=True, how='right')
        n_missing_values = df_data_change.isna().sum().sum()
        df_data_change.dropna(inplace=True)
        
        df_data_change.to_excel('C:/Users/wyd2hu/OneDrive - University of Virginia/Hridita/Predict/Data/'+ds_name+'feature_lebel.xlsx')
        
        print(df_data_change.shape[0])

extract_features()

193
36


# Operation Theatre

In [47]:
ot_df = pd.read_csv("C:/Users/wyd2hu/S2He/AudData/Tiles19/tiles-phase2-opendataset/fitbit/heart-rate/7c78cdc7-915b-4883-9ca2-04c8b8453a38.csv.gz", compression='gzip')
ot_df['Timestamp'] = pd.to_datetime(ot_df['Timestamp'], format="%Y-%m-%dT%H:%M:%S.%f").dt.tz_localize('US/Eastern').dt.strftime("%Y-%m-%d")

In [ ]:
print(len(set(ot_df['Timestamp'])))